# GAWS 明文实现

主要包括以下算法：

- SNP过滤
- PCA
- 优势比
- 趋势卡方计算（检验）

## 1. SNP过滤（略）

## 2. PCA

PCA步骤（设有m条n维数据）：

1. 将原始数据按列组成n行m列矩阵X
2. 将X的每一行（代表一个属性字段）进行零均值化，即减去这一行的均值
3. 求出协方差矩阵
4. 求出协方差矩阵的特征值及对应的特征向量r
5. 将特征向量按对应特征值大小从上到下按行排列成矩阵，取前k行组成矩阵P
6. 即为降维到k维后的数据

In [1]:
import numpy as np
import pandas as pd
from sklearn import decomposition
# load data
data_file = './data/pca-data.txt'
data = pd.read_csv(data_file, sep='\t', index_col=False)
data.set_index('Unnamed: 0', inplace=True)
print(data.iloc[:5, :5])

# pca
print("\n===========使用sklearn库实现的PCA===========")
pca = decomposition.PCA(n_components=5)
pca.fit(data.iloc[:, 1:]) # 每一行代表一个样本
ex_ratio = pca.explained_variance_ratio_
print("ex_ratio: ", ex_ratio)
print("fit_ratio: ", sum(ex_ratio))
print("==========================================")


def pca(data_mat, topNfeat=9999999):
    mean_vals = np.mean(data_mat, axis=0)                 # 求均值
    mean_removed = data_mat - mean_vals                   # 2. 去平均值

    cov_mat = np.cov(mean_removed, rowvar=0)              # 3. 求协方差矩阵(若rowvar非0，一列代表一个样本)

    eig_vals, eig_vects = np.linalg.eig(np.mat(cov_mat))  # 求特征值&特征向量

    eig_val_ind = np.argsort(eig_vals)                    # 从小到大对N个值排序，返回其索引
    eig_val_ind = eig_val_ind[:-(topNfeat+1):-1]
    red_eig_vects = eig_vects[:, eig_val_ind]

    low_d_data_mat = np.dot(mean_removed, red_eig_vects)  # 将数据转换到新空间
    # 重构数据
    recon_mat = (low_d_data_mat * red_eig_vects.T) + mean_vals[:, np.newaxis].T

    return low_d_data_mat, recon_mat, eig_vals

print("\n============基于numpy实现PCA算法============")
low_d_data_mat, recon_mat, eig_vals = pca(data.iloc[:, 1:], topNfeat=5)

low_d_data_mat = pd.DataFrame(low_d_data_mat)
print("PCA降维结果： \n", low_d_data_mat.iloc[:5, :])

recon_mat = pd.DataFrame(recon_mat)
print("\n重构后的数据： \n", recon_mat.iloc[:5, :4])

print("==========================================")

            Case(1)/Control(0)  rs11252546  rs7909677  rs10904494  rs11591988
Unnamed: 0                                                                   
NA18532                      1           0          0           0           1
NA18605                      1           0          0           0           0
NA18542                      1           0          0           0           2
NA18550                      1           0          0           0           0
NA18608                      1           0          1           0           1

===========使用sklearn库实现的PCA===========
ex_ratio:  [0.68845653 0.13193044 0.05248616 0.0383473  0.02685684]
fit_ratio:  0.9380772745916852

============基于numpy实现PCA算法============
PCA降维结果： 
                           0                         1  \
0  (-0.8374473590558991+0j)  (-0.7260946569335854+0j)   
1    (2.303549326998102+0j)  (-0.4159829409654667+0j)   
2  (-2.5671892018087057+0j)   (0.5423241872257525+0j)   
3    (2.303549326998102+0j)  (-0.415

## 3. 优势比

|SNP      | 碱基A | 碱基T |
|:-------:|:----:|:-----:|
|Case组   |a     |b      |
|Control组|c     |d      |

对于次要等位碱基T来说：$T = \frac{bc}{da}$

OR 一般用于计算次要等位基因：
- OR值=1，表示该因素对疾病的发生不起作用。
- OR值>1，表示该因素是一个危险因素
- OR值<1，表示该因素是一个保护因素

## 4. 趋势卡方计算（检验）

### 原理：

如下面$2\times3$的列联表。其中，$R_1=N_{11}+N_{12}+N_{13}$，$C_1=N_{11}+N_{21}$，etc.

|         |B=1      |B=2      |B=3      |sum      |
|:-------:|:-------:|:-------:|:-------:|:-------:|
|A=1      |$N_{11}$ |$N_{12}$ |$N_{13}$ |$R_1$    |
|A=2      |$N_{21}$ |$N_{22}$ |$N_{23}$ |$R_2$    |
|sum      |$C_1$    |$C_2$    |$C_3$    |$N$      |

首先计算：①$T=\sum_{i=1}^k t_{i}(N_{1i}R_2 - N_{2i}R_1)$

然后计算②方差: $Var(T) = \frac{R_1 R_2}{N}(\sum_{i=1}^{k} t_i^2 C_i(N-C_i) - 2\sum_{i=1}^{k-1}\sum_{j=i+1}^{k}t_i t_j C_i C_j)$

最后计算: ③  $\frac{T}{\sqrt{Var(T)}}$

注：对于上面权重 $t$ 的选择，GWAS中一般选择$(1, 1, 0)$ / $(0, 1, 1)$ / $(0, 1, 2)$

### 例子:

假设有如下列联表：

|         | Genotype aa | Genotype Aa | Genotype AA | Sum |
|:-------:|:-----------:|:-----------:|:-----------:|:---:|
|Controls | 20          | 20          | 20          |60   |
|Cases    | 10          | 20          | 30          |60   |
|Sum      | 30          | 40          | 50          |120  |

计算得到：
- T = 600
- Var(T) = 105000.0
- score = 1.85...

*详见以下程序实现*

In [2]:
import math
import numpy as np
class CA_test_for_trend:
    def __init__(self, n_1, n_2, k=3, t=[1, 1, 0]):
        assert len(n_1) == k and len(n_2) == k, "len(n) must equals to k"
        assert len(t) == k, "len(k) must equals to k"
        self.k = k
        self.t = t
        self.table = np.array([n_1, n_2])
        self.R = self.table.sum(axis=1)
        self.C = self.table.sum(axis=0)
        self.N = self.table.sum()
    
    # 计算 T 值
    def cal_T(self):
        T = 0
        for i in range(self.k):
            T += self.t[i] * ( self.table[0][i] * self.R[1] - self.table[1][i] * self.R[0])
        return T

    # 计算 T 的方差
    def cal_varT(self):
        part_1 = self.R[0] * self.R[1] / self.N
        part_2_left = 0
        for i in range(self.k):
            part_2_left += self.t[i]**2 * self.C[i] * (self.N - self.C[i])

        part_2_right = 0
        for i in range(self.k):
            for j in range(i+1, self.k):
                part_2_right += self.t[i] * self.t[j] * self.C[i] * self.C[j]
        else:
            part_2_right *= 2

        return part_1 * (part_2_left - part_2_right)
    
    def cal_score(self, T, var_T):
        return T / math.sqrt(var_T)

# ex:
Controls = [20, 20, 20]
Cases = [10, 20, 30]
K = 3
t = [1, 1, 0]
myca = CA_test_for_trend(Controls, Cases, K, t)
T = myca.cal_T()
var_T = myca.cal_varT()
score = myca.cal_score(T, var_T)
print("T: {}\nVar(T): {}\nscore: {}".format(T, var_T, score))

T: 600
Var(T): 105000.0
score: 1.8516401995451028
